In [ ]:
import numpy as np
import pandas as pd
import uproot
import xgboost as xgb
import matplotlib.pyplot as plt
from xgbo import XgboRegressor
import os
from scipy.stats import norm
import matplotlib.mlab as mlab
import iminuit
import probfit
import probfit.pdf

In [ ]:
def load_data(file_name, entrystop, isEE=False):

    root_file = uproot.open(file_name)

    # The branches we need for the regression
    branches_EB = [ 'clusterRawEnergy', 'full5x5_e3x3', 'full5x5_eMax',
            'full5x5_e2nd', 'full5x5_eTop', 'full5x5_eBottom', 'full5x5_eLeft',
            'full5x5_eRight', 'full5x5_e2x5Max', 'full5x5_e2x5Top',
            'full5x5_e2x5Bottom', 'full5x5_e2x5Left', 'full5x5_e2x5Right',
            'full5x5_e5x5', 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
            'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
            'full5x5_sigmaIphiIphi', 'iEtaSeed', 'iPhiSeed', 'iEtaMod5',
            'iPhiMod2', 'iEtaMod20', 'iPhiMod20', 'genEnergy']

    branches_EE = [ 'clusterRawEnergy', 'full5x5_e3x3', 'full5x5_eMax',
            'full5x5_e2nd', 'full5x5_eTop', 'full5x5_eBottom', 'full5x5_eLeft',
            'full5x5_eRight', 'full5x5_e2x5Max', 'full5x5_e2x5Top',
            'full5x5_e2x5Bottom', 'full5x5_e2x5Left', 'full5x5_e2x5Right',
            'full5x5_e5x5', 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
            'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
            'full5x5_sigmaIphiIphi',
            'genEnergy', 'iXSeed', 'iYSeed', 'preshowerEnergy']

    if isEE:
        branches = branches_EE + ["pt", "eta"]
    else:
        branches = branches_EB + ["pt", "eta"]

    if "Electron" in file_name:
        df = root_file['een_analyzer/ElectronTree'].pandas.df(branches, entrystop=entrystop).dropna()
    if "Photon" in file_name:
        df = root_file['een_analyzer/PhotonTree'].pandas.df(branches, entrystop=entrystop).dropna()
    print("Entries in ntuple:")
    print(len(df))
    print(df.shape)


    df.eval("clusertRawEnergyOverE5x5 = clusterRawEnergy/full5x5_e5x5", inplace=True)
    df.eval("w3x3OverE5x5             = full5x5_e3x3/full5x5_e5x5", inplace=True)
    df.eval("eMaxOverE5x5             = full5x5_eMax/full5x5_e5x5", inplace=True)
    df.eval("e2ndOverE5x5             = full5x5_e2nd/full5x5_e5x5", inplace=True)
    df.eval("eTopOverE5x5             = full5x5_eTop/full5x5_e5x5", inplace=True)
    df.eval("eBottomOverE5x5          = full5x5_eBottom/full5x5_e5x5", inplace=True)
    df.eval("eLeftOverE5x5            = full5x5_eLeft/full5x5_e5x5", inplace=True)
    df.eval("eRightOverE5x5           = full5x5_eRight/full5x5_e5x5", inplace=True)
    df.eval("e2x5MaxOverE5x5          = full5x5_e2x5Max/full5x5_e5x5", inplace=True)
    df.eval("e2x5TopOverE5x5          = full5x5_e2x5Top/full5x5_e5x5", inplace=True)
    df.eval("e2x5BottomOverE5x5       = full5x5_e2x5Bottom/full5x5_e5x5", inplace=True)
    df.eval("e2x5LeftOverE5x5         = full5x5_e2x5Left/full5x5_e5x5", inplace=True)
    df.eval("e2x5RightOverE5x5        = full5x5_e2x5Right/full5x5_e5x5", inplace=True)

    if isEE:
        df.eval("preshowerEnergyOverrawEnergy = preshowerEnergy/rawEnergy", inplace=True)

    # The target
    if isEE:
        df.eval("target = ( rawEnergy + preshowerEnergy )/genEnergy", inplace=True)
    else:
        df.eval("target = rawEnergy/genEnergy", inplace=True)

    return df


In [ ]:
# The features
features_EB = [ 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
        'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
        'full5x5_sigmaIphiIphi', 'clusertRawEnergyOverE5x5', 'w3x3OverE5x5',
        'eMaxOverE5x5', 'e2ndOverE5x5', 'eTopOverE5x5', 'eBottomOverE5x5',
        'eLeftOverE5x5', 'eRightOverE5x5', 'e2x5MaxOverE5x5',
        'e2x5TopOverE5x5', 'e2x5BottomOverE5x5', 'e2x5LeftOverE5x5',
        'e2x5RightOverE5x5', 'iEtaSeed', 'iPhiSeed', 'iEtaMod5', 'iPhiMod2',
        'iEtaMod20', 'iPhiMod20','pt','eta','target']

# EE
features_EE = [ 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
        'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
        'full5x5_sigmaIphiIphi', 'clusertRawEnergyOverE5x5', 'w3x3OverE5x5',
        'eMaxOverE5x5', 'e2ndOverE5x5', 'eTopOverE5x5', 'eBottomOverE5x5',
        'eLeftOverE5x5', 'eRightOverE5x5', 'e2x5MaxOverE5x5',
        'e2x5TopOverE5x5', 'e2x5BottomOverE5x5', 'e2x5LeftOverE5x5',
        'e2x5RightOverE5x5', 'iXSeed', 'iYSeed', 'preshowerEnergyOverrawEnergy','pt','eta','target']


file_name = "/scratch/micheli/Electron/perfectIC-lowpt-EB-training.root"


isEE = '-EE-' in file_name

if isEE:
    features = features_EE
else:
    features = features_EB
DF= load_data(file_name, entrystop=1500000,isEE=isEE)



In [ ]:
DF=DF.query('pt>20')

In [ ]:
df_test  = load_data(file_name, isEE=isEE, entrystop=1500000)
X_test  = df_test[features_EB]
y_test  = df_test["target"]
xgtest  = xgb.DMatrix(X_test , label=y_test)
out_dir='/work/okiss/SemesterProject_2/xgbo/Energy_Regression_test/'
xgbo_default = xgb.Booster({'nthread':4}) #init model
#xgbo_default.load_model("Photon_perfectIC-highpt-EB/model_default/model.bin") # load data
xgbo_default.load_model(out_dir+"Electron_perfectIC-lowpt-EB_test20190124/model_default/model.bin")
xgbo_bo = xgb.Booster({'nthread':4}) #init model
#xgbo_bo.load_model("Photon_perfectIC-highpt-EB/model_optimized/model.bin") # load data
xgbo_bo.load_model(out_dir+"Electron_perfectIC-lowpt-EB_test20190124/model_optimized/model.bin") # load data

preds_default = xgbo_default.predict(xgtest)
preds_bo      = xgbo_bo.predict(xgtest)

# preds = 1.

# Etrue / Eraw after applying corrections
z_default = preds_default / (df_test['genEnergy']/df_test['rawEnergy'])
z_bo      = preds_bo / (df_test['genEnergy']/df_test['rawEnergy'])

bins = np.linspace(0.0, 2.0, 200)
bins_zoom = np.linspace(0.9, 1.1, 200)

plt.figure()
plt.hist(df_test['rawEnergy']/df_test['genEnergy'], bins=bins, histtype='step', label="uncorrected")
plt.hist(z_default, bins=bins, histtype='step', label='corrected (default)')
plt.hist(z_bo, bins=bins, histtype='step', label='corrected (optimized)')
plt.legend()
plt.savefig(os.path.join(out_dir+'plot','energy_comparaison'+'.png'))
plt.show()


plt.figure()
plt.hist(df_test['rawEnergy']/df_test['genEnergy'], bins=bins_zoom, histtype='step', label="uncorrected")
plt.hist(z_default, bins=bins_zoom, histtype='step', label='corrected (default)')
plt.hist(z_bo, bins=bins_zoom, histtype='step', label='corrected (optimized)')
plt.legend()
plt.savefig(os.path.join(out_dir+'plot','energy_comparaison_zoom'+'.png'))
plt.show()

In [ ]:
data=pd.DataFrame()
data['eta']=df_test['eta']
data['pt']=df_test['pt']
data['target']=df_test['target']
data['z_default']=z_default
data['z_bo']=z_bo
print(data.columns)

In [ ]:
n=29
mean_pt_un=np.zeros((n))
mean_err_pt_un=np.zeros((n))
sigma_pt_un=np.zeros((n))
sigma_err_pt_un=np.zeros((n))

pt=np.zeros((n))
feat_pt=[]
for i in range (n):
    a=300-10*i
    b=a+10
    pt[i]= (a+b)/2
    feat_pt.append(str(a)+'<pt<'+str(b))
nbins=200


In [ ]:
i=0
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.95, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(5):
        #first step
       
    if j==0: 
            pars_dcb = dict(mean  = 1.003, 
                        fix_mean = False,
                        sigma  = 0.01,
                        fix_sigma =False,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = False,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 5,
                        fix_n2  = False)
       
    elif j==1:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = 0.1,
                        fix_sigma = True,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
    
    elif j==2:
            pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = True,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = True
                        )
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
    
    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=1
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.93, 1.035]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(5):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0015, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 100,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
  
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=2
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.935, 1.035]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(5):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.002, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 100,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
  
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=3
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.925, 1.035]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
  
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=4
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.93, 1.045]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0005, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
  
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=5
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0005, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=6
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0005, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=7
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0005, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=8
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0005, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=9
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

OK


In [ ]:
i=8
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(5):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.0005, 
                        fix_mean = False,
                        sigma  = 0.011,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 3,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=10
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.945, 1.025]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

# OK after that

In [ ]:
i=11
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.92, 1.04]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')


In [ ]:
i=12
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.9, 1.04]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=13
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=14
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=15
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=16
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=17
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=18
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=19
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=20
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=21
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.85, 1.05]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=22
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.78, 1.06]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=23
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.78, 1.06]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=24
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.78, 1.06]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=25
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.78, 1.06]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=26
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.7, 1.06]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.002, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=27
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.7, 1.1]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.012,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')

In [ ]:
i=28
df=data.query(feat_pt[i]).astype('double')
df=df['target']
bound_DCB=[0.5, 1.35]
normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
for j in range(4):
        #first step
   
    if j==0: 
            pars_dcb = dict(mean  = 1.001, 
                        fix_mean = False,
                        sigma  = 0.016,
                        fix_sigma =True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 5,
                        fix_n  = True,
                        alpha2 =0.8,
                        fix_alpha2 = False,
                        n2     = 110,
                        fix_n2  = True
        
                           ) 
            
    elif j==2:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       
    
            
        #optimize parameter withoptimized start parameter
    else:
        pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
       

    minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_alpha=(0,100),limit_sigma=(0,1),limit_n=(0,1000),limit_n2=(0,1000),limit_mean=(0.9,1.1), **pars_dcb)
        
    minuit.migrad() #optimized parameter
       
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    
mean_pt_un[i]=minuit.values['mean']
mean_err_pt_un[i]=minuit.errors['mean']
sigma_pt_un[i]=minuit.values['sigma']
sigma_err_pt_un[i]=minuit.errors['sigma']
print(feat_pt[i])
print(i)     
plt.title('target:'+feat_pt[i])
plt.savefig(os.path.join(out_dir+'plot','target:'+feat_pt[i]+'_fit'+'.png'))
plt.show()
np.savetxt('pt_parameter_Uncorrected.csv', (mean_pt_un,sigma_pt_un), delimiter=',')